In [1]:
import os

import mlflow
import psycopg2 as psycopg
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error


In [2]:

def fill_missing_values(data):
    cols_with_nans = data.isnull().sum()
    cols_with_nans = cols_with_nans[cols_with_nans > 0].index
    for col in cols_with_nans:
        if data[col].dtype in [float, int]:
            fill_value = data[col].median()
        elif data[col].dtype == 'object':
            fill_value = data[col].mode().iloc[0]
        data[col] = data[col].fillna(fill_value)
    return data 


connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20240325_31df1ce58e",
    "user": "mle_20240325_31df1ce58e",
    "password": "9a3bb87504284307b17eeafcc286313f",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определяем название таблицы, в которой хранятся наши данные
TABLE_NAME = "clean_realty_price"


# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций с базой данных
# причём закрыто оно будет даже в случае ошибки при работе с базой данных
# это нужно, чтобы не допустить так называемую "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаём объект курсора для выполнения запросов к базе данных 
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
				
				# извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

				# получаем список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаём объект DataFrame из полученных данных и имён столбцов 
# это позволяет удобно работать с данными в Python с использованием библиотеки Pandas
df = pd.DataFrame(data, columns=columns)
df=fill_missing_values(df)

df=df.drop(['id','flat_id'],axis=1)
cat_features = df.select_dtypes(include=['object','category']).columns.to_list()

num_features = df.select_dtypes(['float','int']).columns.to_list()


X_train, X_test, y_train, y_test = train_test_split(df.drop(['price'],axis=1), df['price'])


model = CatBoostRegressor(cat_features=cat_features,verbose=False)


model.fit(X_train,y_train)
predictions= model.predict(X_test)




os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
RUN_NAME = "base_model_registry"
REGISTRY_MODEL_NAME = "base_realty_price_model_dvskorokhodov"
YOUR_NAME = "DANIIL_S" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "введите своё имя в переменной YOUR_NAME для создания уникального эксперимента"
EXPERIMENT_NAME = f"{YOUR_NAME}_base_model_registry_3"
#EXPERIMENT_NAME = f"ML_experiment_{YOUR_NAME}_2"
 
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 
try:
  experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
except AttributeError:
  experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
pip_requirements = './requirements.txt'
signature = mlflow.models.infer_signature(X_test, predictions)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}
metrics = {'mae':mean_absolute_error,
           'mse':mean_squared_error,
           'mape':mean_absolute_percentage_error

          }



prediction = model.predict(X_test)
mae = mean_absolute_error(y_test,prediction)# ваш код здесь #
mse = mean_squared_error(y_test,prediction)# ваш код здесь #
mape = mean_absolute_percentage_error(y_test,prediction)# ваш код здесь #


# запишите значения метрик в словарь
metrics["mae"] = mae
metrics["mse"] = mse
metrics["mape"] = mape
best_params= model.get_all_params()
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    model_info = mlflow.catboost.log_model(
        await_registration_for=60, 
        cb_model=model,
        artifact_path="model",
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        metadata=metadata,
        signature=signature,
        input_example=input_example)
    mlflow.log_params(best_params)
    mlflow.log_metrics(metrics)

/home/mle-user/mle_projects/mle_final_projects/mle-project-sprint-2-v001/.venv_mlflow_project/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'base_realty_price_model_dvskorokhodov' already exists. Creating a new version of thi

In [3]:
df.to_csv('./data/data_mart.csv',index=False)